In [1]:
import torch
from torch import nn

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1116],
        [-0.1113]], grad_fn=<AddmmBackward0>)

In [3]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2434, -0.1100,  0.1309,  0.2301, -0.0763, -0.0255,  0.3154,  0.2030]])), ('bias', tensor([-0.1708]))])


In [4]:
print(type(net[2].bias))

<class 'torch.nn.parameter.Parameter'>


In [5]:
print(net[2].bias)

Parameter containing:
tensor([-0.1708], requires_grad=True)


In [6]:
print(net[2].bias.data)

tensor([-0.1708])


In [7]:
net[2].weight.grad == None

True

In [9]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [10]:
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [11]:
net.state_dict()['2.bias'].data

tensor([-0.1708])

In [12]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.1900, -0.0779,  0.4285, -0.4349],
                      [ 0.2189,  0.3050,  0.2046, -0.3727],
                      [ 0.3837,  0.4397, -0.3495,  0.1509],
                      [-0.0377, -0.4028, -0.3741, -0.1015],
                      [ 0.3191, -0.2989, -0.2267,  0.4002],
                      [-0.3694,  0.4979,  0.2060, -0.4651],
                      [-0.3787,  0.3451, -0.4429,  0.4040],
                      [ 0.3714,  0.0248,  0.3810, -0.3639]])),
             ('0.bias',
              tensor([-0.2834,  0.3825, -0.3300,  0.1202,  0.3156,  0.2674, -0.4279,  0.3169])),
             ('2.weight',
              tensor([[ 0.2434, -0.1100,  0.1309,  0.2301, -0.0763, -0.0255,  0.3154,  0.2030]])),
             ('2.bias', tensor([-0.1708]))])

In [13]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

In [14]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3138],
        [-0.3138]], grad_fn=<AddmmBackward0>)

In [15]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [16]:
rgnet.state_dict()

OrderedDict([('0.block 0.0.weight',
              tensor([[-0.1676, -0.1082,  0.3848,  0.1115],
                      [ 0.4825, -0.1003,  0.4507, -0.4876],
                      [ 0.3887,  0.0196,  0.0998,  0.4379],
                      [-0.4831, -0.3463,  0.4970, -0.4265],
                      [-0.3282,  0.4777,  0.1045,  0.3084],
                      [ 0.3728,  0.2287,  0.2448, -0.4413],
                      [ 0.3964, -0.3567,  0.1648,  0.4491],
                      [ 0.2305,  0.4192, -0.0365,  0.0910]])),
             ('0.block 0.0.bias',
              tensor([ 0.0405, -0.0970, -0.4696,  0.0927, -0.2291,  0.3653, -0.4676, -0.2367])),
             ('0.block 0.2.weight',
              tensor([[-2.6223e-01,  2.8297e-01,  2.0721e-01,  5.2101e-02,  1.4441e-01,
                       -1.6125e-01, -2.7731e-01, -2.7768e-01],
                      [ 2.5065e-01,  1.7438e-01, -2.9555e-01, -2.4495e-01,  1.6165e-02,
                        7.8023e-02,  1.4550e-01,  1.5711e-02],
            

In [17]:
rgnet[0][1][0].bias.data

tensor([ 0.3267,  0.0437,  0.3332,  0.1665, -0.1838, -0.4190, -0.3403, -0.1900])

# 参数初始化

In [18]:
 def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0036,  0.0143, -0.0080, -0.0018]), tensor(0.))

In [19]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [20]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

In [21]:
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.6265, -0.2868,  0.3628, -0.1430])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [22]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

In [23]:
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000, -0.0000,  0.0000],
        [-9.7373, -8.8427, -0.0000, -0.0000]], grad_fn=<SliceBackward0>)

In [24]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

# 参数绑定

In [25]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)

tensor([[-0.1004],
        [-0.1164]], grad_fn=<AddmmBackward0>)

In [27]:
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
